# M/M/c Queue Modeling: Analytical and Simulation Approaches

**Author:** fergmlx  
**Date:** 2025-12-09  
**Course:** Operations Research - Queue Theory

---

## Overview

This notebook explores M/M/c queueing systems through:
1. **Analytical formulas** - Exact mathematical solutions
2. **SimPy simulation** - Discrete-event simulation
3. **Comparative analysis** - Validation and insights
4. **Sensitivity analysis** - Impact of parameter changes

### M/M/c Queue Characteristics
- **M** (Markovian/Exponential): Interarrival times
- **M** (Markovian/Exponential): Service times
- **c**: Number of parallel servers

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import simpy
from scipy.special import factorial
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")

---

## 1. Analytical M/M/c Formulas

### Key Performance Metrics

For an M/M/c queue with arrival rate λ and service rate μ:

1. **Traffic intensity**: $\rho = \frac{\lambda}{c\mu}$
2. **Probability of zero customers**: $P_0$
3. **Erlang-C formula**: $C(c, \lambda/\mu)$ - Probability of waiting
4. **Average queue length**: $L_q = C(c, \lambda/\mu) \cdot \frac{\rho}{1-\rho}$
5. **Average waiting time in queue**: $W_q = \frac{L_q}{\lambda}$
6. **Average time in system**: $W = W_q + \frac{1}{\mu}$
7. **Average customers in system**: $L = \lambda W$

In [ ]:
class MMcQueue:
    """
    Analytical M/M/c Queue Model
    
    Parameters:
    -----------
    arrival_rate : float
        Average arrival rate (λ) - customers per unit time
    service_rate : float
        Average service rate per server (μ) - customers per unit time
    num_servers : int
        Number of parallel servers (c)
    """
    
    def __init__(self, arrival_rate, service_rate, num_servers):
        self.lam = arrival_rate
        self.mu = service_rate
        self.c = num_servers
        
        # Validate stability condition
        if arrival_rate >= num_servers * service_rate:
            raise ValueError(f"System is unstable: λ={arrival_rate} >= c*μ={num_servers * service_rate}")
        
        self.rho = arrival_rate / (num_servers * service_rate)  # Utilization
        self.a = arrival_rate / service_rate  # Offered load
        
    def calculate_P0(self):
        """Calculate probability of zero customers in system"""
        sum_term = sum([(self.a ** n) / factorial(n) for n in range(self.c)])
        last_term = (self.a ** self.c) / (factorial(self.c) * (1 - self.rho))
        P0 = 1 / (sum_term + last_term)
        return P0
    
    def erlang_c(self):
        """Calculate Erlang-C formula (probability of waiting)"""
        P0 = self.calculate_P0()
        numerator = (self.a ** self.c) * P0
        denominator = factorial(self.c) * (1 - self.rho)
        return numerator / denominator
    
    def calculate_metrics(self):
        """Calculate all performance metrics"""
        P0 = self.calculate_P0()
        Pc = self.erlang_c()
        
        # Queue metrics
        Lq = Pc * self.rho / (1 - self.rho)  # Average queue length
        Wq = Lq / self.lam  # Average waiting time in queue
        
        # System metrics
        W = Wq + (1 / self.mu)  # Average time in system
        L = self.lam * W  # Average customers in system
        
        # Server utilization
        utilization = self.rho
        
        return {
            'P0': P0,
            'Erlang_C': Pc,
            'Lq': Lq,
            'Wq': Wq,
            'L': L,
            'W': W,
            'Utilization': utilization,
            'rho': self.rho,
            'offered_load': self.a
        }
    
    def print_metrics(self):
        """Display metrics in a formatted table"""
        metrics = self.calculate_metrics()
        
        print(f"\n{'='*60}")
        print(f"M/M/{self.c} Queue - Analytical Results")
        print(f"{'='*60}")
        print(f"Arrival rate (λ):          {self.lam:.4f} customers/time")
        print(f"Service rate (μ):          {self.mu:.4f} customers/time")
        print(f"Number of servers (c):     {self.c}")
        print(f"Offered load (a=λ/μ):      {metrics['offered_load']:.4f}")
        print(f"-" * 60)
        print(f"Server utilization (ρ):    {metrics['Utilization']:.4f} ({metrics['Utilization']*100:.2f}%)")
        print(f"P(0 customers):            {metrics['P0']:.6f}")
        print(f"Erlang-C (P(wait)):        {metrics['Erlang_C']:.6f}")
        print(f"-" * 60)
        print(f"Avg queue length (Lq):     {metrics['Lq']:.4f} customers")
        print(f"Avg system length (L):     {metrics['L']:.4f} customers")
        print(f"Avg wait time (Wq):        {metrics['Wq']:.4f} time units")
        print(f"Avg system time (W):       {metrics['W']:.4f} time units")
        print(f"{'='*60}\n")

# Test the analytical model
print("Analytical M/M/c Queue Model initialized!")

---

## 2. Comparing Different Server Configurations

Let's analyze how system performance changes with different numbers of servers.

In [ ]:
# Define base parameters
lambda_base = 8.0  # 8 customers per hour
mu_base = 2.0      # 2 customers per hour per server

# Test different server configurations
server_configs = [3, 4, 5, 6, 7, 8]
results = []

for c in server_configs:
    try:
        queue = MMcQueue(lambda_base, mu_base, c)
        metrics = queue.calculate_metrics()
        metrics['servers'] = c
        results.append(metrics)
    except ValueError as e:
        print(f"Skipping c={c}: {e}")

# Create DataFrame
df_configs = pd.DataFrame(results)
df_configs = df_configs[['servers', 'Utilization', 'Erlang_C', 'Lq', 'Wq', 'L', 'W']]

print("\nComparison of Server Configurations")
print("=" * 100)
print(df_configs.to_string(index=False))
print("=" * 100)

---

## 3. Visualizing Performance Metrics

Visual comparison of key metrics across different server configurations.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('M/M/c Queue Performance Metrics vs Number of Servers', fontsize=16, fontweight='bold')

# 1. Average Wait Time in Queue (Wq)
axes[0, 0].plot(df_configs['servers'], df_configs['Wq'], 'o-', linewidth=2, markersize=8, color='#e74c3c')
axes[0, 0].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[0, 0].set_ylabel('Avg Wait Time (Wq)', fontweight='bold')
axes[0, 0].set_title('Average Waiting Time in Queue')
axes[0, 0].grid(True, alpha=0.3)

# 2. Average Queue Length (Lq)
axes[0, 1].plot(df_configs['servers'], df_configs['Lq'], 'o-', linewidth=2, markersize=8, color='#3498db')
axes[0, 1].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[0, 1].set_ylabel('Avg Queue Length (Lq)', fontweight='bold')
axes[0, 1].set_title('Average Number in Queue')
axes[0, 1].grid(True, alpha=0.3)

# 3. Server Utilization
axes[0, 2].plot(df_configs['servers'], df_configs['Utilization']*100, 'o-', linewidth=2, markersize=8, color='#2ecc71')
axes[0, 2].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[0, 2].set_ylabel('Utilization (%)', fontweight='bold')
axes[0, 2].set_title('Server Utilization Rate')
axes[0, 2].axhline(y=80, color='orange', linestyle='--', label='80% Target')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Erlang-C (Probability of Waiting)
axes[1, 0].plot(df_configs['servers'], df_configs['Erlang_C']*100, 'o-', linewidth=2, markersize=8, color='#9b59b6')
axes[1, 0].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[1, 0].set_ylabel('Probability of Waiting (%)', fontweight='bold')
axes[1, 0].set_title('Erlang-C: Probability Customer Must Wait')
axes[1, 0].grid(True, alpha=0.3)

# 5. Average System Length (L)
axes[1, 1].plot(df_configs['servers'], df_configs['L'], 'o-', linewidth=2, markersize=8, color='#f39c12')
axes[1, 1].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[1, 1].set_ylabel('Avg System Length (L)', fontweight='bold')
axes[1, 1].set_title('Average Customers in System')
axes[1, 1].grid(True, alpha=0.3)

# 6. Average Time in System (W)
axes[1, 2].plot(df_configs['servers'], df_configs['W'], 'o-', linewidth=2, markersize=8, color='#1abc9c')
axes[1, 2].set_xlabel('Number of Servers (c)', fontweight='bold')
axes[1, 2].set_ylabel('Avg Time in System (W)', fontweight='bold')
axes[1, 2].set_title('Average Total Time in System')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("- As servers increase, wait time and queue length decrease exponentially")
print("- Utilization decreases with more servers (diminishing returns)")
print("- There's a trade-off between service level and resource efficiency")

---

## 4. SimPy Discrete-Event Simulation

Now let's implement a SimPy simulation to validate our analytical results.

In [ ]:
class MMcSimulation:
    """
    SimPy-based M/M/c Queue Simulation
    
    Parameters:
    -----------
    arrival_rate : float
        Average arrival rate (λ)
    service_rate : float
        Average service rate per server (μ)
    num_servers : int
        Number of parallel servers (c)
    sim_time : float
        Total simulation time
    warmup_time : float
        Warmup period to exclude from statistics
    """
    
    def __init__(self, arrival_rate, service_rate, num_servers, sim_time=10000, warmup_time=1000):
        self.lam = arrival_rate
        self.mu = service_rate
        self.c = num_servers
        self.sim_time = sim_time
        self.warmup_time = warmup_time
        
        # Statistics collectors
        self.wait_times = []
        self.system_times = []
        self.queue_lengths = []
        self.system_lengths = []
        self.customers_served = 0
        self.total_wait_time = 0
        
    def customer(self, env, name, servers):
        """Process representing a customer"""
        arrival_time = env.now
        
        # Record queue length at arrival (only after warmup)
        if arrival_time >= self.warmup_time:
            self.queue_lengths.append(len(servers.queue))
            self.system_lengths.append(servers.count + len(servers.queue))
        
        # Request a server
        with servers.request() as request:
            yield request
            
            wait_time = env.now - arrival_time
            
            # Generate service time
            service_time = np.random.exponential(1.0 / self.mu)
            yield env.timeout(service_time)
            
            system_time = env.now - arrival_time
            
            # Record statistics (only after warmup)
            if arrival_time >= self.warmup_time:
                self.wait_times.append(wait_time)
                self.system_times.append(system_time)
                self.customers_served += 1
                self.total_wait_time += wait_time
    
    def arrival_process(self, env, servers):
        """Generate customer arrivals"""
        customer_count = 0
        while True:
            # Generate interarrival time
            interarrival_time = np.random.exponential(1.0 / self.lam)
            yield env.timeout(interarrival_time)
            
            # Create new customer
            customer_count += 1
            env.process(self.customer(env, f'Customer{customer_count}', servers))
    
    def run(self, random_seed=42):
        """Run the simulation"""
        np.random.seed(random_seed)
        
        # Create SimPy environment
        env = simpy.Environment()
        
        # Create server resource
        servers = simpy.Resource(env, capacity=self.c)
        
        # Start arrival process
        env.process(self.arrival_process(env, servers))
        
        # Run simulation
        env.run(until=self.sim_time)
        
        return self.calculate_metrics()
    
    def calculate_metrics(self):
        """Calculate performance metrics from simulation data"""
        if self.customers_served == 0:
            return None
        
        Wq_sim = np.mean(self.wait_times)
        W_sim = np.mean(self.system_times)
        Lq_sim = np.mean(self.queue_lengths)
        L_sim = np.mean(self.system_lengths)
        
        # Calculate utilization (customers served * avg service time / (servers * sim time))
        actual_sim_time = self.sim_time - self.warmup_time
        utilization_sim = (self.customers_served / self.mu) / (self.c * actual_sim_time)
        
        # Probability of waiting (proportion with wait time > 0)
        prob_wait = sum(1 for w in self.wait_times if w > 0.001) / len(self.wait_times)
        
        return {
            'Wq': Wq_sim,
            'W': W_sim,
            'Lq': Lq_sim,
            'L': L_sim,
            'Utilization': utilization_sim,
            'Prob_Wait': prob_wait,
            'Customers_Served': self.customers_served
        }

print("SimPy M/M/c Simulation Model initialized!")

---

## 5. Running Simulation for c=5 Servers

Let's run a detailed simulation with 5 servers and compare with analytical results.

In [ ]:
# Configuration for c=5 servers
c_selected = 5
lambda_val = 8.0
mu_val = 2.0

print(f"\n{'='*70}")
print(f"Running Simulation: M/M/{c_selected} Queue")
print(f"{'='*70}")
print(f"Parameters:")
print(f"  - Arrival rate (λ): {lambda_val} customers/hour")
print(f"  - Service rate (μ): {mu_val} customers/hour/server")
print(f"  - Number of servers: {c_selected}")
print(f"  - Simulation time: 10,000 time units")
print(f"  - Warmup time: 1,000 time units")
print(f"\nRunning simulation... ", end='')

# Run analytical model
analytical = MMcQueue(lambda_val, mu_val, c_selected)
analytical_metrics = analytical.calculate_metrics()

# Run simulation
simulation = MMcSimulation(lambda_val, mu_val, c_selected, sim_time=10000, warmup_time=1000)
sim_metrics = simulation.run(random_seed=42)

print("Complete!\n")

# Display analytical results
analytical.print_metrics()

---

## 6. Analytical vs Simulation Comparison

Detailed comparison to validate simulation accuracy.

In [ ]:
# Create comparison table
comparison_data = {
    'Metric': ['Avg Wait Time (Wq)', 'Avg System Time (W)', 'Avg Queue Length (Lq)', 
               'Avg System Length (L)', 'Utilization (ρ)', 'Prob. of Waiting'],
    'Analytical': [
        analytical_metrics['Wq'],
        analytical_metrics['W'],
        analytical_metrics['Lq'],
        analytical_metrics['L'],
        analytical_metrics['Utilization'],
        analytical_metrics['Erlang_C']
    ],
    'Simulation': [
        sim_metrics['Wq'],
        sim_metrics['W'],
        sim_metrics['Lq'],
        sim_metrics['L'],
        sim_metrics['Utilization'],
        sim_metrics['Prob_Wait']
    ]
}

df_comparison = pd.DataFrame(comparison_data)
df_comparison['Difference (%)'] = abs(df_comparison['Analytical'] - df_comparison['Simulation']) / df_comparison['Analytical'] * 100

print(f"\n{'='*90}")
print(f"Analytical vs Simulation Comparison (c={c_selected})")
print(f"{'='*90}")
print(df_comparison.to_string(index=False))
print(f"{'='*90}")
print(f"\nSimulation Details:")
print(f"  - Customers served: {sim_metrics['Customers_Served']:,}")
print(f"  - Mean absolute error: {df_comparison['Difference (%)'].mean():.2f}%")
print(f"\nConclusion: {'Excellent' if df_comparison['Difference (%)'].mean() < 5 else 'Good' if df_comparison['Difference (%)'].mean() < 10 else 'Fair'} agreement between analytical and simulation results!")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot comparison
metrics_to_plot = ['Avg Wait Time (Wq)', 'Avg Queue Length (Lq)', 'Utilization (ρ)', 'Prob. of Waiting']
indices = [0, 2, 4, 5]
x = np.arange(len(metrics_to_plot))
width = 0.35

analytical_vals = [df_comparison.iloc[i]['Analytical'] for i in indices]
simulation_vals = [df_comparison.iloc[i]['Simulation'] for i in indices]

axes[0].bar(x - width/2, analytical_vals, width, label='Analytical', color='#3498db', alpha=0.8)
axes[0].bar(x + width/2, simulation_vals, width, label='Simulation', color='#e74c3c', alpha=0.8)
axes[0].set_xlabel('Metric', fontweight='bold')
axes[0].set_ylabel('Value', fontweight='bold')
axes[0].set_title(f'Analytical vs Simulation: M/M/{c_selected} Queue', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels([m.split('(')[0].strip() for m in metrics_to_plot], rotation=15, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Error percentage plot
colors = ['green' if x < 5 else 'orange' if x < 10 else 'red' for x in df_comparison['Difference (%)']]
axes[1].barh(df_comparison['Metric'], df_comparison['Difference (%)'], color=colors, alpha=0.7)
axes[1].set_xlabel('Difference (%)', fontweight='bold')
axes[1].set_title('Percentage Difference: |Analytical - Simulation|', fontweight='bold')
axes[1].axvline(x=5, color='orange', linestyle='--', linewidth=1, label='5% threshold')
axes[1].axvline(x=10, color='red', linestyle='--', linewidth=1, label='10% threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---

## 7. Distribution Analysis from Simulation

Examine the distributions of wait times and queue lengths from simulation.

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Wait time distribution
axes[0, 0].hist(simulation.wait_times, bins=50, density=True, alpha=0.7, color='#3498db', edgecolor='black')
axes[0, 0].axvline(analytical_metrics['Wq'], color='red', linestyle='--', linewidth=2, label=f"Analytical: {analytical_metrics['Wq']:.3f}")
axes[0, 0].axvline(sim_metrics['Wq'], color='green', linestyle='--', linewidth=2, label=f"Simulation: {sim_metrics['Wq']:.3f}")
axes[0, 0].set_xlabel('Wait Time (Wq)', fontweight='bold')
axes[0, 0].set_ylabel('Density', fontweight='bold')
axes[0, 0].set_title('Distribution of Wait Times in Queue', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# System time distribution
axes[0, 1].hist(simulation.system_times, bins=50, density=True, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[0, 1].axvline(analytical_metrics['W'], color='red', linestyle='--', linewidth=2, label=f"Analytical: {analytical_metrics['W']:.3f}")
axes[0, 1].axvline(sim_metrics['W'], color='green', linestyle='--', linewidth=2, label=f"Simulation: {sim_metrics['W']:.3f}")
axes[0, 1].set_xlabel('System Time (W)', fontweight='bold')
axes[0, 1].set_ylabel('Density', fontweight='bold')
axes[0, 1].set_title('Distribution of Total Time in System', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Queue length distribution
queue_counts = pd.Series(simulation.queue_lengths).value_counts().sort_index()
axes[1, 0].bar(queue_counts.index, queue_counts.values / len(simulation.queue_lengths), 
               alpha=0.7, color='#2ecc71', edgecolor='black')
axes[1, 0].axvline(analytical_metrics['Lq'], color='red', linestyle='--', linewidth=2, label=f"Analytical: {analytical_metrics['Lq']:.3f}")
axes[1, 0].axvline(sim_metrics['Lq'], color='blue', linestyle='--', linewidth=2, label=f"Simulation: {sim_metrics['Lq']:.3f}")
axes[1, 0].set_xlabel('Queue Length (Lq)', fontweight='bold')
axes[1, 0].set_ylabel('Probability', fontweight='bold')
axes[1, 0].set_title('Distribution of Queue Length', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# System length distribution
system_counts = pd.Series(simulation.system_lengths).value_counts().sort_index()
axes[1, 1].bar(system_counts.index, system_counts.values / len(simulation.system_lengths), 
               alpha=0.7, color='#f39c12', edgecolor='black')
axes[1, 1].axvline(analytical_metrics['L'], color='red', linestyle='--', linewidth=2, label=f"Analytical: {analytical_metrics['L']:.3f}")
axes[1, 1].axvline(sim_metrics['L'], color='blue', linestyle='--', linewidth=2, label=f"Simulation: {sim_metrics['L']:.3f}")
axes[1, 1].set_xlabel('System Length (L)', fontweight='bold')
axes[1, 1].set_ylabel('Probability', fontweight='bold')
axes[1, 1].set_title('Distribution of Customers in System', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDistribution Statistics:")
print(f"  Wait Time - Mean: {np.mean(simulation.wait_times):.4f}, Std: {np.std(simulation.wait_times):.4f}")
print(f"  System Time - Mean: {np.mean(simulation.system_times):.4f}, Std: {np.std(simulation.system_times):.4f}")
print(f"  Queue Length - Mean: {np.mean(simulation.queue_lengths):.4f}, Std: {np.std(simulation.queue_lengths):.4f}")
print(f"  System Length - Mean: {np.mean(simulation.system_lengths):.4f}, Std: {np.std(simulation.system_lengths):.4f}")

---

## 8. Sensitivity Analysis on Arrival Rates

Examine how system performance changes as arrival rate increases (keeping service rate and servers constant).

In [ ]:
# Sensitivity analysis parameters
c_fixed = 5
mu_fixed = 2.0
lambda_range = np.linspace(1.0, 9.5, 20)  # Max is 10 (c*μ) for stability

sensitivity_results = []

print("\nPerforming sensitivity analysis on arrival rate...")
print(f"Fixed parameters: c={c_fixed}, μ={mu_fixed}")
print(f"Testing λ from {lambda_range[0]:.1f} to {lambda_range[-1]:.1f}\n")

for lam in lambda_range:
    try:
        queue = MMcQueue(lam, mu_fixed, c_fixed)
        metrics = queue.calculate_metrics()
        metrics['lambda'] = lam
        metrics['rho_percent'] = metrics['Utilization'] * 100
        sensitivity_results.append(metrics)
    except ValueError:
        print(f"Skipping λ={lam:.2f} (unstable system)")
        break

df_sensitivity = pd.DataFrame(sensitivity_results)

print(f"Completed {len(sensitivity_results)} analysis points.")

In [ ]:
# Visualize sensitivity analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Sensitivity Analysis: Impact of Arrival Rate (λ) on M/M/{c_fixed} Queue', 
             fontsize=16, fontweight='bold')

# 1. Wait Time vs Arrival Rate
axes[0, 0].plot(df_sensitivity['lambda'], df_sensitivity['Wq'], 'o-', linewidth=2, markersize=6, color='#e74c3c')
axes[0, 0].set_xlabel('Arrival Rate (λ)', fontweight='bold')
axes[0, 0].set_ylabel('Avg Wait Time (Wq)', fontweight='bold')
axes[0, 0].set_title('Wait Time Increases Rapidly Near Capacity')
axes[0, 0].axvline(x=c_fixed*mu_fixed, color='red', linestyle='--', alpha=0.5, label=f'Capacity (c·μ={c_fixed*mu_fixed})')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# 2. Queue Length vs Arrival Rate
axes[0, 1].plot(df_sensitivity['lambda'], df_sensitivity['Lq'], 'o-', linewidth=2, markersize=6, color='#3498db')
axes[0, 1].set_xlabel('Arrival Rate (λ)', fontweight='bold')
axes[0, 1].set_ylabel('Avg Queue Length (Lq)', fontweight='bold')
axes[0, 1].set_title('Queue Length Grows Exponentially')
axes[0, 1].axvline(x=c_fixed*mu_fixed, color='red', linestyle='--', alpha=0.5, label=f'Capacity (c·μ={c_fixed*mu_fixed})')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# 3. Utilization vs Arrival Rate
axes[1, 0].plot(df_sensitivity['lambda'], df_sensitivity['rho_percent'], 'o-', linewidth=2, markersize=6, color='#2ecc71')
axes[1, 0].set_xlabel('Arrival Rate (λ)', fontweight='bold')
axes[1, 0].set_ylabel('Utilization (%)', fontweight='bold')
axes[1, 0].set_title('Server Utilization Increases Linearly')
axes[1, 0].axhline(y=80, color='orange', linestyle='--', alpha=0.7, label='80% Target')
axes[1, 0].axhline(y=90, color='red', linestyle='--', alpha=0.7, label='90% High Load')
axes[1, 0].axvline(x=c_fixed*mu_fixed, color='red', linestyle='--', alpha=0.5)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# 4. Erlang-C vs Arrival Rate
axes[1, 1].plot(df_sensitivity['lambda'], df_sensitivity['Erlang_C']*100, 'o-', linewidth=2, markersize=6, color='#9b59b6')
axes[1, 1].set_xlabel('Arrival Rate (λ)', fontweight='bold')
axes[1, 1].set_ylabel('Probability of Waiting (%)', fontweight='bold')
axes[1, 1].set_title('Probability Customer Must Wait')
axes[1, 1].axvline(x=c_fixed*mu_fixed, color='red', linestyle='--', alpha=0.5, label=f'Capacity (c·μ={c_fixed*mu_fixed})')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Create summary table for key arrival rates
key_lambdas = [4.0, 6.0, 8.0, 9.0, 9.5]
summary_data = []

for lam in key_lambdas:
    try:
        queue = MMcQueue(lam, mu_fixed, c_fixed)
        metrics = queue.calculate_metrics()
        summary_data.append({
            'λ': lam,
            'ρ (%)': f"{metrics['Utilization']*100:.1f}",
            'Wq': f"{metrics['Wq']:.4f}",
            'Lq': f"{metrics['Lq']:.4f}",
            'P(wait)': f"{metrics['Erlang_C']*100:.2f}%"
        })
    except ValueError:
        summary_data.append({
            'λ': lam,
            'ρ (%)': 'UNSTABLE',
            'Wq': '-',
            'Lq': '-',
            'P(wait)': '-'
        })

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*70)
print(f"Sensitivity Analysis Summary: M/M/{c_fixed} with μ={mu_fixed}")
print("="*70)
print(df_summary.to_string(index=False))
print("="*70)

print("\n📊 Key Insights from Sensitivity Analysis:")
print("\n1. **Non-linear Response**: As arrival rate approaches capacity (c·μ=10),")
print("   wait times and queue lengths increase exponentially.")
print("\n2. **Critical Threshold**: Around 80-85% utilization, system performance")
print("   begins to degrade significantly.")
print("\n3. **Stability Boundary**: System becomes unstable when λ ≥ c·μ.")
print("\n4. **Practical Recommendation**: Operate below 80% utilization for")
print("   acceptable service levels and system stability.")

---

## 9. Multiple Replications Analysis

Run multiple simulation replications to assess variability and confidence.

In [ ]:
# Run multiple replications
num_replications = 30
c_test = 5
lambda_test = 8.0
mu_test = 2.0

print(f"\nRunning {num_replications} simulation replications...")
print(f"Configuration: M/M/{c_test}, λ={lambda_test}, μ={mu_test}\n")

replication_results = []

for i in range(num_replications):
    sim = MMcSimulation(lambda_test, mu_test, c_test, sim_time=10000, warmup_time=1000)
    metrics = sim.run(random_seed=i)
    metrics['replication'] = i + 1
    replication_results.append(metrics)
    
    if (i + 1) % 10 == 0:
        print(f"  Completed {i + 1}/{num_replications} replications")

df_replications = pd.DataFrame(replication_results)

print("\nReplication analysis complete!")

In [ ]:
# Statistical analysis of replications
stats_summary = df_replications[['Wq', 'Lq', 'W', 'L', 'Utilization']].describe()

print("\n" + "="*80)
print("Statistical Summary of Simulation Replications")
print("="*80)
print(stats_summary)
print("="*80)

# Calculate 95% confidence intervals
from scipy import stats as scipy_stats

confidence_level = 0.95
alpha = 1 - confidence_level

print(f"\n95% Confidence Intervals (n={num_replications} replications):")
print("-" * 80)

for metric in ['Wq', 'Lq', 'W', 'L', 'Utilization']:
    data = df_replications[metric]
    mean = data.mean()
    sem = scipy_stats.sem(data)
    ci = scipy_stats.t.interval(confidence_level, len(data)-1, loc=mean, scale=sem)
    
    print(f"{metric:15s}: {mean:.6f} ± {(ci[1]-mean):.6f}  [{ci[0]:.6f}, {ci[1]:.6f}]")

print("-" * 80)

# Compare with analytical
analytical_comp = MMcQueue(lambda_test, mu_test, c_test)
analytical_metrics_comp = analytical_comp.calculate_metrics()

print("\nComparison with Analytical Results:")
print("-" * 80)
for metric in ['Wq', 'Lq', 'W', 'L', 'Utilization']:
    sim_mean = df_replications[metric].mean()
    analytical_val = analytical_metrics_comp[metric]
    diff_pct = abs(sim_mean - analytical_val) / analytical_val * 100
    
    print(f"{metric:15s}: Analytical={analytical_val:.6f}, Simulation={sim_mean:.6f}, Diff={diff_pct:.2f}%")
print("-" * 80)

In [ ]:
# Visualize replication variability
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Variability Across {num_replications} Simulation Replications', fontsize=16, fontweight='bold')

# Box plots for key metrics
metrics_to_plot = [('Wq', 'Avg Wait Time'), ('Lq', 'Avg Queue Length'), 
                   ('W', 'Avg System Time'), ('Utilization', 'Utilization')]

for idx, (metric, label) in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    # Box plot
    bp = ax.boxplot([df_replications[metric]], labels=[label], patch_artist=True)
    bp['boxes'][0].set_facecolor('#3498db')
    bp['boxes'][0].set_alpha(0.7)
    
    # Add analytical line
    analytical_val = analytical_metrics_comp[metric]
    ax.axhline(y=analytical_val, color='red', linestyle='--', linewidth=2, label='Analytical')
    
    # Add mean line
    sim_mean = df_replications[metric].mean()
    ax.axhline(y=sim_mean, color='green', linestyle='--', linewidth=2, label='Sim Mean')
    
    ax.set_ylabel(label, fontweight='bold')
    ax.set_title(f'{label} Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Replication analysis shows consistent results across multiple runs.")
print("   The narrow confidence intervals indicate high precision in our estimates.")

---

## 10. Conclusions and Recommendations

### Key Findings

1. **Model Validation**: Simulation results closely match analytical predictions, validating both approaches.

2. **Server Configuration**: 
   - More servers reduce wait times but increase costs
   - Optimal configuration balances service level and resource utilization
   - Target utilization: 70-80% for good service with efficiency

3. **Sensitivity to Load**:
   - System performance degrades rapidly as utilization exceeds 80%
   - Operating near capacity leads to exponentially increasing wait times
   - Small increases in arrival rate can cause dramatic performance drops

4. **Simulation Insights**:
   - Multiple replications provide confidence in estimates
   - Discrete-event simulation captures operational variability
   - Useful for scenarios where analytical solutions are complex

### Practical Recommendations

1. **Capacity Planning**: Maintain 20-30% capacity buffer above expected demand
2. **Performance Monitoring**: Track utilization and wait times continuously
3. **Dynamic Staffing**: Adjust server count based on demand patterns
4. **Service Level Targets**: Set realistic targets based on cost-service trade-offs

### Future Work

- Explore non-exponential service time distributions (M/G/c queues)
- Implement priority queuing systems
- Analyze time-varying arrival rates
- Optimize staffing schedules with cost constraints

In [ ]:
# Final summary visualization
print("\n" + "="*80)
print("NOTEBOOK EXECUTION COMPLETE")
print("="*80)
print("\n📚 Summary of Analyses Performed:")
print("  ✓ Analytical M/M/c queue formulas implemented")
print("  ✓ Multiple server configurations compared")
print("  ✓ Performance metrics visualized")
print("  ✓ SimPy discrete-event simulation developed")
print("  ✓ Analytical vs simulation validation completed")
print("  ✓ Sensitivity analysis on arrival rates performed")
print("  ✓ Multiple replications for statistical confidence")
print("\n🎯 Key Takeaway: M/M/c queues can be analyzed both analytically and")
print("   through simulation, each providing valuable insights for system design.")
print("\n" + "="*80)